In [57]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd()))

In [58]:
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import ColumnDataSource, HoverTool, Span, BoxSelectTool, LabelSet, LinearColorMapper
from bokeh.transform import transform
from bokeh.palettes import Viridis11
from bokeh.layouts import gridplot, layout
from load_data import load_data
from config import RED

output_file('bokeh_dashboard.html')

In [59]:
df = load_data()

In [60]:
num_offenses_df = (df.resample('M', on='OCCURRED_ON_DATE').SHOOTING
                .agg(num_offenses="count", 
                     shootings="sum")
               )

num_offenses_source = ColumnDataSource(num_offenses_df)

top10_groups = df.OFFENSE_CODE_GROUP.value_counts().iloc[:10].sort_values(ascending=True).reset_index().rename(columns={"index": "code_group", "OFFENSE_CODE_GROUP": "counts"})
top10_source = ColumnDataSource(top10_groups)

per_day_hour = df.assign(HOUR=lambda x: x.HOUR.astype(str)).groupby(['DAY_OF_WEEK', 'HOUR']).size().rename('counts').reset_index()
per_day_hour_source = ColumnDataSource(per_day_hour)

day_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hours = [str(x) for x in range(0, 24)]

In [61]:
offenses_tooltip = HoverTool(
    tooltips=[('Date', '@OCCURRED_ON_DATE{%b %Y}'),
              ('Number of Offenses', '@num_offenses')],
    formatters={'OCCURRED_ON_DATE': 'datetime'},
    mode="vline"
)

num_offenses = figure(x_axis_type="datetime", title="Number of Offenses per Month")
num_offenses.line(x='OCCURRED_ON_DATE', y='num_offenses', source=num_offenses_source, color=RED, line_width=1.5)
num_offenses.add_tools(offenses_tooltip)
num_offenses.circle(x='OCCURRED_ON_DATE', y='num_offenses', source=num_offenses_source, color="black", size=4)
num_offenses.yaxis.axis_label = 'Number of Offenses'
mean_line = Span(location=num_offenses_source.data['num_offenses'].mean(), dimension='width', line_color=RED, line_dash=[8, 3])
num_offenses.add_layout(mean_line)

In [62]:
shootings_tooltip = HoverTool(
    tooltips=[('Date', '@OCCURRED_ON_DATE{%b %Y}'),
              ('Number of Shootings', '@shootings')],
    formatters={'OCCURRED_ON_DATE': 'datetime'},
    mode="vline"
)

num_shootings = figure(x_axis_type="datetime", title="Number of Shootings per Month")
num_shootings.add_tools(shootings_tooltip)
num_shootings.line(x='OCCURRED_ON_DATE', y='shootings', source=num_offenses_source, color=RED, line_width=1.5)
num_shootings.circle(x='OCCURRED_ON_DATE', y='shootings', source=num_offenses_source, color="black", size=4)
num_shootings.yaxis.axis_label = "Number of Shootings"
mean_shootings = Span(location=num_offenses_source.data["shootings"].mean(), dimension="width", line_color=RED, line_dash=[8, 3])
num_shootings.add_layout(mean_shootings)

In [63]:
top10 = figure(y_range=top10_groups.code_group, title="Top 10 Offence Code Groups")
top10.hbar(right="counts", y="code_group", height=0.8, source=top10_source)
labels = LabelSet(x="counts", y="code_group", text="counts", source=top10_source, x_offset=5, y_offset=-10, text_font_size='1em')
top10.add_layout(labels)
top10.plot_width=800

In [64]:
mapper = LinearColorMapper(palette=Viridis11, low=per_day_hour.counts.min(), high=per_day_hour.counts.max())
heatmap = figure(title="Number of Offenses per Hour and Day Of Week", x_range=day_of_week, y_range=hours, tools="hover")
heatmap.hover.tooltips = [("Day of Week", "@DAY_OF_WEEK"),
                         ("Hour of Day", "@HOUR"),
                         ("Number of Offenses", "@counts")]
heatmap.rect(x='DAY_OF_WEEK', y='HOUR', source=per_day_hour_source, width=1, height=1, fill_color=transform('counts', mapper), line_color=None)

heatmap.axis.axis_line_color = None
heatmap.axis.major_tick_line_color = None
heatmap.axis.major_label_text_font_size = "8pt"
heatmap.axis.major_label_standoff = 0

In [65]:
# dashboard = gridplot([num_offenses, num_shootings, top10, heatmap], ncols=2, plot_width=600)

In [69]:
dashboard = layout([[num_offenses, num_shootings], [top10], [heatmap]], sizing_mode='stretch_width')

In [70]:
show(dashboard)